#Paso 1: Instalación de dependencias

In [1]:
!pip install librosa tensorflow resampy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.3 MB/s eta 0:00:00


In [2]:
import os
import librosa
import numpy as np
import tensorflow as tf
from google.colab import files
from google.colab import output
from IPython.display import Audio
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU, BatchNormalization

#Paso 2: Cargar las carpetas de comandos y procesar los audios
Este código recorrerá las carpetas cargadas en Google Colab, donde cada carpeta tiene el nombre del comando de voz correspondiente. Convertimos los archivos WAV en espectrogramas Mel (una representación que permite al modelo aprender mejor).

In [4]:
# Directorio que contiene las carpetas de comandos
data_dir = "/content/comandos/"  # Cambia este path según donde cargues tus carpetas

# Listas para guardar las características y etiquetas
X = []
y = []

# Función para extraer características Mel del audio
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
    return mfccs_scaled

# Cargar los audios y etiquetas
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.wav'):
                file_path = os.path.join(folder_path, file)
                features = extract_features(file_path)
                X.append(features)
                y.append(folder)

# Convertir a arrays y etiquetas numéricas
X = np.array(X)
le = LabelEncoder()
y = le.fit_transform(y)

# Dividir en entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Paso 3: Definir y entrenar el modelo
Este es un modelo básico de red neuronal para clasificar los comandos. Puedes ajustar la arquitectura según sea necesario.

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(256),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(128),
    BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(64),
    BatchNormalization(),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(32),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')  # Capa de salida
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Guardar checkpoints durante el entrenamiento
checkpoint_filepath = '/content/checkpoints/checkpoint.weights.h5'  # Cambiar a .weights.h5
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    verbose=1
)

# Entrenar el modelo
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, callbacks=[checkpoint_callback])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1149 - loss: 2.3341
Epoch 1: val_loss improved from inf to 2.17569, saving model to /content/checkpoints/checkpoint.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.1204 - loss: 2.3348 - val_accuracy: 0.1695 - val_loss: 2.1757
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1984 - loss: 2.1669 
Epoch 2: val_loss improved from 2.17569 to 2.11203, saving model to /content/checkpoints/checkpoint.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1971 - loss: 2.1669 - val_accuracy: 0.1695 - val_loss: 2.1120
Epoch 3/50
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2500 - loss: 1.9082
Epoch 3: val_loss improved from 2.11203 to 2.07091, saving model to /content/checkpoints/checkpoint.weights.h5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.2231 - loss: 2.0318 - val_accuracy: 0.2373 - val_loss: 2.0709
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2345 - loss: 2.0982

#Paso 4: Cargar el modelo desde el último checkpoint y continuar el entrenamiento (opcional).
En caso de que el entrenamiento se interrumpa, puedes cargar el checkpoint guardado y continuar el entrenamiento desde allí.

In [6]:
# Cargar el modelo desde el checkpoint
model.load_weights(checkpoint_filepath)

# Continuar el entrenamiento
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, callbacks=[checkpoint_callback])

Epoch 1/50
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7257 - loss: 0.8170
Epoch 1: val_loss did not improve from 0.88087
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7214 - loss: 0.8201 - val_accuracy: 0.5424 - val_loss: 1.1269
Epoch 2/50
6/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7592 - loss: 0.7885
Epoch 2: val_loss did not improve from 0.88087
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7557 - loss: 0.7849 - val_accuracy: 0.5593 - val_loss: 1.1864
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7123 - loss: 0.7957  
Epoch 3: val_loss did not improve from 0.88087
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7141 - loss: 0.7923 - val_accuracy: 0.6102 - val_loss: 0.9490
Epoch 4/50
7/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7419 - loss: 0.7464
Epoch 4: val_loss did not improve from 0.88087
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7503 - loss: 0.7397 - val_accuracy: 0.6102 - val_loss: 1.0243
Epoch 5/50
7/8 ━━━━━━━━

#Paso 5: Guardar el modelo completo
Después de entrenar, guarda el modelo completo para futuras predicciones.

In [7]:
model.save('/content/comandos_model.h5')

#Paso 6: Cargar el modelo y hacer predicciones
Ahora puedes cargar el modelo y hacer predicciones sobre nuevos archivos de audio.

In [8]:
# Subir un archivo de audio manualmente
uploaded = files.upload()

# Renombrar el archivo subido a 'nuevo_comando.wav'
for filename in uploaded.keys():
    os.rename(filename, '/content/nuevo_comando.wav')

# Verificar si el archivo fue renombrado correctamente
print("Archivo renombrado a: nuevo_comando.wav")

Saving Grabación (1).WAV to Grabación (1).WAV
Archivo renombrado a: nuevo_comando.wav


In [9]:
# Cargar el modelo guardado
model = tf.keras.models.load_model('/content/comandos_model.h5')

# Función para predecir el comando de un archivo de audio
def predict_command(file_path):
    features = extract_features(file_path)
    features = features.reshape(1, -1)
    prediction = model.predict(features)
    predicted_label = np.argmax(prediction, axis=1)
    return le.inverse_transform(predicted_label)[0]

In [10]:
# Asigna la ruta del archivo renombrado
new_audio_path = '/content/nuevo_comando.wav'

# Predecir el comando del nuevo archivo
predicted_command = predict_command(new_audio_path)
print(f"Comando predicho: {predicted_command}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
Comando predicho: Girar


Dependiendo del caso se puede preferir el resultado como un string de numpy que sirve para hacer comparaciones o si se prefiere pasarlo a string de python. Depende de la aplicación.

In [11]:
type(predicted_command)

numpy.str_

In [16]:
if predicted_command=="Derecha" or predicted_command=="Izquierda" or predicted_command=="Girar":
    print("*Procede a girar*")

*Procede a girar*


In [12]:
predicted_output = str(predicted_command)
print(predicted_output)
type(predicted_output)

Girar


str

In [15]:
if predicted_output=="Derecha" or predicted_output=="Izquierda" or predicted_output=="Girar":
    print("*Procede a girar*")

*Procede a girar*
